# Boundary Value Problem

$$
u'' - 2 u' + u = 1, \quad u(0) = 0, \quad u'(1) = 1
$$

---

## Solve the Homogeneous Equation

The homogeneous equation is:

$$
u_h'' - 2 u_h' + u_h = 0
$$

The characteristic equation is:

$$
r^2 - 2 r + 1 = 0 \implies (r-1)^2 = 0
$$

which has a **double root** $r=1$. Therefore, the homogeneous solution is:

$$
u_h(x) = C_1 e^x + C_2 x e^x
$$

where $C_1$ and $C_2$ are constants.

---

## Find a Particular Solution

Since the right-hand side is constant $1$, we try a constant particular solution $u_p = A$.  

Substituting:

$$
u_p'' - 2 u_p' + u_p = 0 - 0 + A = 1 \implies A = 1
$$

So the particular solution is:

$$
u_p(x) = 1
$$

---

## General Solution

The general solution is:

$$
u(x) = u_h(x) + u_p(x) = C_1 e^x + C_2 x e^x + 1
$$

---

## Apply Boundary Conditions

1. Dirichlet condition at $x=0$:

$$
u(0) = C_1 + 0 + 1 = 0 \implies C_1 = -1
$$

2. Neumann condition at $x=1$:

Compute derivative:

$$
u'(x) = C_1 e^x + C_2 e^x + C_2 x e^x = C_1 e^x + C_2 (1+x) e^x
$$

Apply $u'(1) = 1$:

$$
u'(1) = -e + 2 C_2 e = 1 \implies 2 C_2 e = e + 1 \implies C_2 = \frac{1 + 1/e}{2}
$$

---

## Exact Solution

Thus, the exact solution of the boundary value problem is:

$$
u(x) = - e^x + \frac{1+1/e}{2} \, x e^x + 1
$$



In [1]:
import numpy as np

# ================================================================
# Exact solution
# ================================================================
def u_exact(x):
    C1 = -1
    C2 = (1 + 1/np.e)/2
    return C1 * np.exp(x) + C2 * x * np.exp(x) + 1

# ================================================================
# Right-hand side f(x)
# ================================================================
def f(x):
    return np.ones_like(x)  # RHS = 1

# ================================================================
# 2nd-order finite difference solver with 2nd-order Neumann BC
# ================================================================
def solve_bvp_fd_2nd_order(N):
    h = 1/N
    x = np.linspace(0,1,N+1)

    A = np.zeros((N+1,N+1))
    b = f(x)

    # Interior points: u'' - 2u' + u = 1
    for i in range(1,N):
        A[i,i-1] = 1/h**2 + 1/h
        A[i,i]   = -2/h**2 + 1
        A[i,i+1] = 1/h**2 - 1/h

    # Dirichlet BC: u(0)=0
    A[0,:] = 0
    A[0,0] = 1
    b[0] = 0

    # Neumann BC at x=1, 2nd-order backward difference:
    # u'(x_N) ≈ (3 u_N - 4 u_{N-1} + u_{N-2}) / (2 h) = 1
    A[N,N-2] = 1/(2*h)
    A[N,N-1] = -4/(2*h)
    A[N,N]   = 3/(2*h)
    b[N] = 1

    # Solve linear system
    u = np.linalg.solve(A,b)
    return x, u

# ================================================================
# Compute error norms
# ================================================================
def error_norms(N):
    x, u = solve_bvp_fd_2nd_order(N)
    u_ex = u_exact(x)
    err = np.abs(u - u_ex)
    L_inf = np.max(err)
    L2 = np.sqrt(np.sum(err**2)*(1/N))
    return L_inf, L2

# ================================================================
# Convergence table
# ================================================================
Ns = [20,40,80,160,320]
results = []

prev_inf = None
prev_L2 = None

for N in Ns:
    L_inf, L2 = error_norms(N)
    if prev_inf is None:
        rate_inf = rate_L2 = np.nan
    else:
        rate_inf = np.log(prev_inf / L_inf)/np.log(2)
        rate_L2  = np.log(prev_L2 / L2)/np.log(2)
    results.append([N, L_inf, rate_inf, L2, rate_L2])
    prev_inf, prev_L2 = L_inf, L2

# ================================================================
# table
# ================================================================
header = (
    f"{'N':>6} | "
    f"{'L_inf error':>15} | {'rate':>7} | "
    f"{'L2 error':>15} | {'rate':>7}"
)
line = "-"*len(header)

print("\nConvergence Table (2nd-order FD with 2nd-order Neumann BC)")
print(line)
print(header)
print(line)
for N,Linf,rinf,L2,r2 in results:
    print(f"{N:6d} | {Linf:15.8e} | {rinf:7.3f} | {L2:15.8e} | {r2:7.3f}")
print(line)



Convergence Table (2nd-order FD with 2nd-order Neumann BC)
--------------------------------------------------------------
     N |     L_inf error |    rate |        L2 error |    rate
--------------------------------------------------------------
    20 |  1.32777886e-03 |     nan |  5.74786124e-04 |     nan
    40 |  3.43277967e-04 |   1.952 |  1.43884165e-04 |   1.998
    80 |  8.73266385e-05 |   1.975 |  3.60100542e-05 |   1.998
   160 |  2.20260840e-05 |   1.987 |  9.00841387e-06 |   1.999
   320 |  5.53121363e-06 |   1.994 |  2.25290673e-06 |   1.999
--------------------------------------------------------------
